In [1]:
import data
import models
import cache
import pandas as pd
import seaborn as sns

/home/jacquelineburos/miniconda3/envs/immuneinf/lib/python3.5/site-packages/Cython/Distutils/old_build_ext.py:30: UserWarning: Cython.Distutils.old_build_ext does not properly handle dependencies and is deprecated.
  "Cython.Distutils.old_build_ext does not properly handle dependencies "
INFO:stancache.seed:Setting seed to 1245502385
INFO:root:Setting CACHE_DIR = /mnt/modelcache/immune-infiltrate-explorations
INFO:stancache.seed:Setting seed to 1245502385


In [2]:
model_name = 'model6.2'
by = 'SubSet'
sample_n = 500 ## number genes
test_sample_n = 1 ## number out of sample samples
train_sample_n = None

In [3]:
sample_df = cache.cached(models.prep_sample_df,
                         sample_n=sample_n)

(training_df, test_df) = cache.cached(
    models.split_sample_df,
    sample_df=sample_df,
    test_sample_n=test_sample_n,
    train_sample_n=train_sample_n)

INFO:stancache.stancache:prep_sample_df: cache_filename set to prep_sample_df.cached.sample_n_500.pkl
INFO:stancache.stancache:prep_sample_df: Loading result from cache
INFO:stancache.stancache:split_sample_df: cache_filename set to split_sample_df.cached.sample_df_65714237990.test_sample_n_1.train_sample_n_82429761595.pkl
INFO:stancache.stancache:split_sample_df: Loading result from cache


In [4]:
model_file = models.get_model_file(model_name=model_name)
#print(cache._read_file(model_file))

In [5]:
stan_data = models.prep_stan_data(sample_df=training_df, test_df=test_df, by=by)

In [ ]:
model_fit = models.cached_stan_fit(file=model_file,
                                   data=stan_data,
                                   model_name=model_name,
                                   iter=50000)

INFO:stancache.stancache:Step 1: Get compiled model code, possibly from cache
INFO:stancache.stancache:StanModel: cache_filename set to model6_2.cython_0_25_1.model_code_15105066910483198602.pystan_2_12_0_0.stanmodel.pkl
INFO:stancache.stancache:StanModel: Loading result from cache
INFO:stancache.stancache:Step 2: Get posterior draws from model, possibly from cache
INFO:stancache.stancache:sampling: cache_filename set to model6_2.cython_0_25_1.model_code_15105066910483198602.pystan_2_12_0_0.stanfit.chains_4.data_85710450591.iter_50000.seed_1245502385.pkl
INFO:stancache.stancache:sampling: Starting execution


In [ ]:
models.print_stan_summary(model_fit, pars='lp__')

In [ ]:
models.print_stan_summary(model_fit, pars='sample2_x')

In [ ]:
colnames = list(stan_data['x'].columns)
colnames

In [ ]:
inferred_type = models.extract_theta_summary(colnames=colnames, par='sample2_x', stan_fit=model_fit)

In [ ]:
df = pd.melt(inferred_type, id_vars='iter', value_name='estimate', var_name='SubSet')

In [ ]:
sns.boxplot(data=df, y='SubSet', x='estimate')

In [ ]:
test_df['SubSet'].unique()